In [2]:
import os
#import keras
import librosa
from librosa import util

import numpy as np
import pandas as pd
import random as rnd
import time
rnd.seed(int(time.time())) # generate seed from the time at which this script is run
import scipy
from scipy import signal
import sys

import librosa.display
from math import pi
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'librosa'

In [ ]:
""" functions creating different types of noise """    
def white_noise(x, SNR):
    N = max(x.shape);
    # N = len(x) alternatively
    sigma = np.sqrt( (x @ x.T) / (N * 10**(SNR/10)) )
    noise = [sigma * rnd.uniform(-1,1) for k in range( N) ]
    return noise

def pink_noise(x, SNR):
    """Generates pink noise using the Voss-McCartney algorithm.
        
    nrows: number of values to generate
    rcols: number of random sources to add
    
    returns: NumPy array
    """
    nrows = len(x) #x.shape
    ncols=16
    
    array = np.empty((nrows, ncols))
    array.fill(np.nan)
    array[0, :] = np.random.random(ncols)
    array[:, 0] = np.random.random(nrows)
    
    # the total number of changes is nrows
    n = nrows
    cols = np.random.geometric(0.5, n)
    cols[cols >= ncols] = 0
    rows = np.random.randint(nrows, size=n)
    array[rows, cols] = np.random.random(n)

    df = pd.DataFrame(array)
    df.fillna(method='ffill', axis=0, inplace=True)
    total = df.sum(axis=1)

    sigma = np.sqrt( (x @ x.T) / (nrows * 10**(SNR/10)) )
    noise= sigma*(total.values-np.mean(total.values)) / (max(total.values) - np.mean(total.values))
    
    return noise

In [ ]:
#loading librosa example
y, sr = librosa.load(librosa.util.example_audio_file(),sr=16000)
N = len(y)
t = np.arange(0,N/sr,1/sr)

# noise params
SNR = 0
sigma = np.sqrt(y@(y.T)/(N*10**(SNR/10)));

# ennoising
yest = y + [sigma* rnd.uniform(-.1,.1) for k in range(len(t)) ]



In [ ]:
# plots
plt.subplot(2,1,1), plt.plot(y[:10000],'b'), 
plt.subplot(2,1,2),plt.plot(yest[:10000],'r'), plt.show()
plt.plot(y[:10000]-yest[:10000])

In [ ]:
def thirdoct(fs, N_fft, numBands, mn):
      # %   [A CF] = THIRDOCT(FS, N_FFT, NUMBANDS, MN) returns 1/3 octave band matrix
      # %   inputs:
      # %       FS:         samplerate 
      # %       N_FFT:      FFT size
      # %       NUMBANDS:   number of bands
      # %       MN:         center frequency of first 1/3 octave band
      # %   outputs:
      # %       A:          octave band matrix
      # %       CF:         center frequencies

    f = np.linspace(0, fs, N_fft+1)
    f = f[0:np.int(N_fft/2+1)]
    print('f.shape: {0}'.format(f.shape[0]))
    #plt.plot(f), plt.show()

    cf  = np.zeros(numBands)
    fl  = np.zeros(numBands)
    fr  = np.zeros(numBands)


    for k in range(numBands):
        fl[k]  = np.sqrt((2**(k/3)*mn)* 2**((k-1)/3)*mn)
        fr[k]  = np.sqrt((2**(k/3)*mn)* 2**((k+1)/3)*mn)
        cf[k]  = 2**(k/3)*mn
    print('fl.shape: {0}'.format(fl.shape))
    #plt.plot(fl), plt.plot(fr), plt.show()

    A = np.zeros((numBands, f.shape[0]))

    for i in range(cf.shape[0]):
        deltaL = (f-fl[i])**2
        #plt.plot(deltaL)
        fl_ii = np.where(deltaL==np.min(deltaL))
        fl[i]= f[fl_ii]
        #print('L, where: {0} '.format(fl_ii, fl[i]))

        deltaR = (f-fr[i])**2
        fr_ii                   = np.where(deltaR== np.min(deltaR))
        fr[i]                   = f[fr_ii]
        #print('R, where: {0} and how much: {1}'.format(fr_ii, fr[i]))

        A[i,fl_ii[0][0]:(fr_ii[0][0])]= 1
    print('A.shape: {0}'.format(A.shape))

    rnk         = np.sum(A, axis=1)
    print('rnk.shape: {0}'.format(rnk.shape))

    selecBands  	= ((rnk[1:]>=rnk[:-1]) * (rnk[1:]!=0)!=0) *range(rnk.shape[0]-1);
    numBands = selecBands[selecBands!=0][-1] + 2 #+1 for add the missing term in row 48, +1 to get the proper number of bands from the dimension 

    print('numBands: {0}'.format(numBands))

    A           = A[:numBands, :]
    cf          = cf[:numBands]
    #print('type(A): {0}'.format(type(A)))
    return A, cf

In [ ]:
n_fft = 512
numBands = 15
mn = 150
STOI_sr = 1e4

[ A, cf] = thirdoct(STOI_sr, n_fft, numBands, mn)

In [ ]:
def removeSilentFrames(x, y, rg, N, K):
#   %   [X_SIL Y_SIL] = REMOVESILENTFRAMES(X, Y, RANGE, N, K) X and Y
#   %   are segmented with frame-length N and overlap K, where the maximum energy
#   %   of all frames of X is determined, say X_MAX. X_SIL and Y_SIL are the
#   %   reconstructed signals, excluding the frames, where the energy of a frame
#   %   of X is smaller than X_MAX-RANGE

    K = np.int(K)
    frames  = list(range(0,x.shape[0]-N, K))
    w       = np.hanning(N)
    msk     = np.zeros(len(frames))
  
#   print('len(frames): {0}'.format(len(frames)))
#   print('(frames): {0}'.format(frames))
#   print('frames[end]: {0}'.format(frames[-1]))
  
    for j in range(len(frames)):
        jj      = np.linspace(frames[j], frames[j]+N-1,N).astype(int)
        msk[j]	= 20*np.log10(np.linalg.norm(x[jj]*[ww for ww in w]) /np.sqrt(N))
  
    #print('max[msk]: {0}'.format(max(msk)))
    msk     = (msk-max(msk)+rg)>0;
    count   = 0;

    x_sil   = np.zeros(x.shape);
    y_sil   = np.zeros(y.shape);
  

    for j in range(len(frames)):
        if msk[j]:
            jj_i            = range(np.int(frames[j]),np.int(frames[j])+N);
            jj_o            = range(np.int(frames[count]),np.int(frames[count])+N);
            x_sil[jj_o]     = x_sil[jj_i] + x[jj_i]*[ww for ww in w];
            y_sil[jj_o]  	= y_sil[jj_i] + y[jj_i]*[ww for ww in w];
            count           = count+1;

    return x_sil, y_sil

In [ ]:
n_fft = 256
STOI_dyn_range = 40
[yR, yestR] = removeSilentFrames(y, yest, STOI_dyn_range, n_fft, n_fft/2)

In [ ]:
def taa_corr(x, y):
#   %   RHO = TAA_CORR(X, Y) Returns correlation coefficient between column
#   %   vectors x and y. Gives same results as 'corr' from statistics toolbox.
    xn    = x-np.mean(x)
    xn  	= xn/np.sqrt(np.sum(xn**2))
    yn   	= y-np.mean(y)
    yn    = yn/np.sqrt(np.sum(yn**2))
    rho   = np.sum(xn*yn)
    return rho

In [ ]:
def calc_STOI(y, yest, sr, **kwargs):
    """
    References:
%      [1] C.H.Taal, R.C.Hendriks, R.Heusdens, J.Jensen 'A Short-Time
%      Objective Intelligibility Measure for Time-Frequency Weighted Noisy
%      Speech', ICASSP 2010, Texas, Dallas.
%
%      [2] C.H.Taal, R.C.Hendriks, R.Heusdens, J.Jensen 'An Algorithm for 
%      Intelligibility Prediction of Time-Frequency Weighted Noisy Speech', 
%      IEEE Transactions on Audio, Speech, and Language Processing, 2011. 
    
    """
    
    
    if y.shape[0] != yest.shape[0]:
      print('y and yest should have the same length')
    print('length of y: {0}'.format(y.shape))
    
    #### Initialisation
    keys = kwargs.keys()
    
    STOI_sr = kwargs.pop('STOI_sr', '') if 'STOI_sr' in keys else 10000
    win_length = kwargs.pop('win_length','') if 'win_length' in keys else 256 
    n_fft = kwargs.pop('n_fft','') if 'n_fft' in keys else 512 # 0-padded to 512
    fBands = kwargs.pop('fBands','') if 'fBands' in keys else 15 # cf literature
    minFreqCenter = kwargs.pop('minFreqCenter','') if 'minFreqCenter' in keys else 150 #en Hz, cf literature    
    hop_length = kwargs.pop('STOI_hop_length','') if 'STOI_hop_length' in keys else np.int(win_length/2) 
    
    H = thirdoct(STOI_sr, n_fft, fBands, minFreqCenter)[0] # Calculate the 1/3 octaves decomposition
    print('type(H): {0} and H.shape: {1}'.format(type(H), H.shape))
    
    STOIframe_t = kwargs.pop('STOIframe_t','') if 'STOIframe_t' in keys else .400 # in sec, optimal for STOI, according to the ref
    STOIframe_n = np.int(STOIframe_t * STOI_sr)
    print('STOIframe_n: {0}'.format(STOIframe_n))
    N = np.int(STOIframe_n / hop_length) # .400/(512 / sr) = 30.0
    print('nbFramesToGetA384ms_longFrame: {0}'.format(N))
    
    beta = kwargs.pop('beta', '') if 'beta' in keys else -15 #lower SDR bound
    STOI_dyn_range = kwargs.pop('STOI_dyn_range','') if 'STOI_dyn_range' in keys else 40;  
    
    #### Resampling    
    if STOI_sr != sr:
        print('Resampling from {} to {}'.format(sr, STOI_sr))
        y = librosa.core.resample(y, sr, STOI_sr)
        yest = librosa.core.resample(yest, sr, STOI_sr)
    print('dim of y post resampling: {0}'.format(y.shape))
  
    #### Find sequences without speech (energy < 40 dB) and eliminate them
    [y, yest] = removeSilentFrames(y, yest, STOI_dyn_range, win_length, hop_length)
    print('new dimensions of y (post removing silemt frames): {0}'.format(y.shape))

  
    #### stft of time-domain signals / can be done with specs but then includes uncertainties about the parameters' values used
    Y    = librosa.core.stft(y,    n_fft=n_fft, hop_length= hop_length, win_length=win_length, window='hann')
    Y_dB = librosa.core.amplitude_to_db(np.abs(Y))
    Y_power = librosa.core.db_to_power(Y_dB, ref=1.0)
    
    print('dimensions of Y: {0}'.format(Y.shape))

    Yest = librosa.core.stft(yest, n_fft=n_fft, hop_length= hop_length, win_length=win_length, window='hann')
    Yest_dB = librosa.core.amplitude_to_db(np.abs(Yest))
    Yest_power = librosa.core.db_to_power(Yest_dB, ref=1.0)
    
        
       

    ##### calculate T-F units
    Y_TF_units = np.empty(( fBands, Y_power.shape[1])) 
    Yest_TF_units = np.empty(( fBands, Y_power.shape[1])) 

    for t in range(Y_power.shape[1]):
        Y_TF_units[:,t]    = np.sqrt(H @ Y_power[:,t]) 
        Yest_TF_units[:,t] = np.sqrt(H @ Yest_power[:,t]) 
        
    print('Yest_TF_units.shape :{0}'.format(Yest_TF_units.shape))

    ##### Short-term segments: group N TF-units to create 400ms(ish)-long frames
    n_overlap = np.int(N*.5)
    totNbTimeFrames = 1+np.int((Yest.shape[1] - N) / (N-n_overlap)) # np.int((Yest.shape[1] - N) / (STOIframe_n-1))
    print('totNbTimeFrames (with overlap): {0}'.format(totNbTimeFrames) )

    # loop all segments of length N and obtain intermediate intelligibility measure for all TF-regions
    d_interm  	= np.zeros((fBands, len(range(N, Y_TF_units.shape[1]))))         # init memory for intermediate intelligibility measure
    c           = 10**(-beta/20);                                                # constant for clipping procedure

    for m in range(N,Y_TF_units.shape[1]): #(N,N+1): 
        Y_seg  	 = Y_TF_units[:, (m-N):m]                                        # region with length N of clean TF-units for all j
        Yest_seg = Yest_TF_units[:, (m-N):m]
        
        if m == 3*N:
            print('Y_seg.shape : {0}'.format(Y_seg.shape ) )

        alpha   = np.sqrt(np.sum(Y_seg**2, axis=1) / (np.sum(Yest_seg**2, axis=1) + sys.float_info.epsilon) )       # obtain scale factor for normalizing processed TF-region for all j

        aYest_seg 	= Yest_seg.T * np.tile(alpha, [N, 1])                        # obtain \alpha*Y_j(n) from Eq.(2) [1]
#         print('aYest_seg.shape {0}'.format(aYest_seg.shape ) )
        aYest_seg = aYest_seg.T
#         print('aYest_seg.shape {0}'.format(aYest_seg.shape ) )
#         OR: aYest_seg 	= [Yest_seg[:,j] * alpha for j in range(Yest_seg.shape[1])] 
        
        for j in range(fBands):
            Yest_prime = [np.min((aYest_el, Y_el*(1+c))) for aYest_el, Y_el in zip(aYest_seg[j,:], Y_seg[j,:])]# apply clipping from Eq.(3)   	
            
            if j == 0 and m == 3*N:
#                 print(' aYest_seg[j, :]{0}'.format( aYest_seg[j, :] ) )
#                 print('Y_seg[j, :]*(1+c) : {0}'.format(Y_seg[j, :]*(1+c)) )        
                plt.subplot(3,1,1),plt.plot(aYest_seg[j,:])
                plt.subplot(3,1,2),plt.plot(Y_seg[j,:]*(1+c)), plt.show()
                print(' Yest_prime.type : {0}'.format( type(Yest_prime)) )
#                 print(' Yest_prime.shape : {0}'.format( Yest_prime.shape) )
                print('Y_seg[j, :].T.shape: {0}'.format(Y_seg[j, :].T.shape ) )
                plt.subplot(3,1,3),plt.plot(Yest_prime)
                
            d_interm[j, m-N]  = taa_corr(Y_seg[j, :].T, Yest_prime)          # obtain correlation coeffecient from Eq.(4) [1]
    
    STOI = np.mean(d_interm)              
    print('shape(d_interm): {0}'.format(d_interm.shape)  )
    print('mean(d_interm,0): {0}'.format(np.mean(d_interm,axis=0))  )
    print('mean(d_interm,1): {0}'.format(np.mean(d_interm,axis=1))  )
    
    
    print('STOI: {0}'.format(STOI) )
    
    print('0: non-intelligible, 1: very intelligible')
    return STOI

In [ ]:
stoi=calc_STOI(y, yest, sr)